
# Datasets currently not used



## [The COVID tracking project testing data.](#source1)
[https://covidtracking.com/api](https://covidtracking.com/api)

## [Delphi-epidata ](#delphi) which contains 
       Facebook surveys, google surveys, doctor visits, google health trends, quidel test data
[https://cmu-delphi.github.io/delphi-epidata/api/covidcast.html](https://cmu-delphi.github.io/delphi-epidata/api/covidcast.html)

I have not dove into this dataset too thoroughly but it contains information from facebook and google
surveys regarding COVID as well as doctor visits; the doctor visit data attempts to make distinctions between
those sick with the annual influenza and those with COVID.


## [IHME hospital data](#ihme)

**Data available at:**
[http://www.healthdata.org/covid/data-downloads](http://www.healthdata.org/covid/data-downloads)

The IHME hospital data is one of the more unique datasets I've discovered with 



## IHME hospital data
<a id='ihme'></a>
[Return to table of contents](#toc)

[JHU CSSE](#csse) 
<font color='red'>
### Has all USA states but only 32 countries which overlap with other data; stash this dataset for now. 
</font>


In [ ]:
ihme_df = column_or_index_string_reformat(pd.read_csv(
    './IHME_hospital_data/2020_04_12.02/Hospitalization_all_locs.csv').rename(columns={'location_name':'location'}))
ihme_df.loc[:, 'date'] = pd.to_datetime(ihme_df.loc[:,'date']).dt.normalize()
ihme_df = ihme_df.set_index(['location', 'date']).sort_index()
ihme_df.sample(5)

## Delphi-epidata
<a id='delphi'></a>
[Return to table of contents](#toc)

data_source	name of upstream data source 
(e.g., fb-survey, google-survey, ght, quidel, doctor-visits)	string

signal	name of signal derived from upstream data (see notes below)	string

time_type	temporal resolution of the signal (e.g., day, week)	string

geo_type	spatial resolution of the signal (e.g., county, hrr, msa, dma, state)	string

time_values	time unit (e.g., date) over which underlying events happened	list of time values (e.g., 20200401)

geo_value	unique code for each location, depending on geo_type (county -> FIPS 6-4 code, HRR -> HRR number, MSA -> CBSA code,
DMA -> DMA code, state -> two-letter state code), or * for all	string

As of this writing, data sources have the following signals:

fb-survey signal values include raw_cli, raw_ili, raw_wcli, raw_wili, and also four additional named with raw_* replaced by smoothed_* (e.g. smoothed_cli, etc).
google-survey signal values include raw_cli and smoothed_cli.
ght signal values include raw_search and smoothed_search.
quidel signal values include smoothed_pct_negative and smoothed_tests_per_device.
doctor-visits signal values include smoothed_cli.

Delphi API data :
doctor visits : 20200201-20200429 (as of 20200503)


In [ ]:
#----------------- Currently Unused ----------------------#

def pull_delphi_data(data_source=['fb-survey', 'google-survey', 'ght', 'quidel', 'quidelneg', 'doctor-visits'], 
                     daterange=pd.date_range(start="20200101",
                                             end=''.join(str(datetime.now().date()).split('-'))).strftime('%Y%m%d'),
                     **kwargs):
    """ Pull data from https://cmu-delphi.github.io/delphi-epidata/api/
        https://cmu-delphi.github.io/delphi-epidata/api/covidcast.html
    
    
    
    """
    
    for data in data_source:
        signal_dict = {'fb-survey':'smoothed_cli',
                       'google-survey':'smoothed_cli',
                       'ght':'smoothed_search',
                       'quidel':'smoothed_tests_per_device',
                       'quidelneg':'smoothed_pct_negative',
                       'doctor-visits':'smoothed_cli'}
        
        signal = signal_dict[data]
        if data=='quidelneg':
            #change the proxy for the quidel signal
            data = 'quidel'
        for days in daterange:
            resp = requests.get('https://delphi.cmu.edu/epidata/api.php?source=covidcast&data_source=doctor-visits&signal=smoothed_cli&time_type=day&geo_type=county&geo_value=*&time_values='+days)
            day_data = resp.json().get('epidata', None)
            if day_data is None:
                pass
            else:
                var_number += pd.json_normalize(day_data).size
                print(pd.json_normalize(day_data).shape)    
                
                
# date_range_2020 = pd.date_range(start="20200101",end=''.join(str(datetime.now().date()).split('-'))).strftime('%Y%m%d')
# var_number = 0 
# for days in date_range_2020:
# #     days='20200302'
#     resp = requests.get('https://delphi.cmu.edu/epidata/api.php?source=covidcast&data_source=doctor-visits&signal=smoothed_cli&time_type=day&geo_type=county&geo_value=*&time_values='+days)
#     day_data = resp.json().get('epidata', None)
#     if day_data is None:
#         pass
#     else:
#         var_number += pd.json_normalize(day_data).size
#         print(pd.json_normalize(day_data).shape)

# <font color='red'>Unused as of now</font>


## Repeat of the above calculations for United States only data.



The United States' data merits separate investigation 1. because of the case number 2. because the IHME dataset is only really
properly defined for the statewide description of the U.S. 

In [ ]:
csse_us_timeseries_df.groupby(level=1).sum().to_csv('csse_united_states.csv')

usa_data = [
    csse_us_timeseries_df,
    ihme_df,
    owid_df,
    oxcgrt_df,
    testtrack_df]

us_table = multiindex_to_table(csse_us_timeseries_df)
today = us_table[us_table.date==us_table.date.max()]

In [ ]:
# unused
#Pull the data using their API (for whatever reason this data set is different from the manual download).
# url_to_present_date = 'https://covidtrackerapi.bsg.ox.ac.uk/api/v2/stringency/date-range/2020-01-02/' \
#                         + str(datetime.now().date())
# response = requests.get(url_to_present_date)
# response_json = response.json()
# response_json_nested_dict = response_json['data']

# response_api_df = pd.DataFrame.from_dict({(i,j): response_json_nested_dict[i][j] 
#                            for i in response_json_nested_dict.keys() 
#                            for j in response_json_nested_dict[i].keys()},
#                        orient='index')